### Analysis of Indel Recombitron data 


In [1]:
import os
from Bio.SeqIO import QualityIO
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import gzip
import glob
import re
from DMS_utils import dna_rev_comp, translate_dna2aa
import pysam
import pandas as pd
import seaborn as sns
import pickle as pkl
import matplotlib.colors as mcolors
from scipy import stats
import os.path
from matplotlib.lines import Line2D
import json
import shutil

In [ ]:
### first, we need to run CRISPResso2 on the data to get the indel frequencies ###
# we use the linker positions (retrons are constructed to create indels only at these sites) to represent "Cas9 cut sites" --> even though we dont use the Cas9 system, our system induces similar patterns in the sequences, thus we can perform the same analysis whereby the linker sequence represent the guide RNA
# the command (in the command line) is for instance:

#### for the forward read only (only first 150 positions of the LOV gene (+beginning of amplicon))

# CRISPResso --fastq_r1 RL1_R1_001.fastq --amplicon_seq CGCCGCATGGAAGCGATTAACGAAAGCAGCGGTTTAGCCACAACGCTGGAACGCATTGAAAAGAATTTCGTAATCACAGACCCGCGCCTTCCCGACAATCCAATTATTTTTGCGTCCGATAGCTTCCTGCAATTAACCGAATACAGCCGCGAAGAAATTCTGGGTCGTAATTGTCGCTTCCTT -n RL1-R1_crispresso_result --guide_seq GAAGCGATTAACGAAAGCAGCGGT --cleavage_offset 0 --amplicon_min_alignment_score 50 --min_average_read_quality 20 --min_bp_quality_or_N 20  --offset_around_cut_to_plot 25 --window_around_sgrna 10  --min_frequency_alleles_around_cut_to_plot 0.0001 --needleman_wunsch_gap_incentive -20 --needleman_wunsch_gap_extend 0


#### for both (R1 and R2), however due to low coverage, many reads are not aligned well enough and thus excluded from the analysis

# CRISPResso --fastq_r1 RL1_R1_001.fastq --fastq_r2 RL1_R2_001.fastq --amplicon_seq CGCCGCATGGAAGCGATTAACGAAAGCAGCGGTTTAGCCACAACGCTGGAACGCATTGAAAAGAATTTCGTAATCACAGACCCGCGCCTTCCCGACAATCCAATTATTTTTGCGTCCGATAGCTTCCTGCAATTAACCGAATACAGCCGCGAAGAAATTCTGGGTCGTAATTGTCGCTTCCTTCAGGGGCCAGAGACTGACCGTGCTACGGTACGCAAAATCCGCGACGCAATCGACAATCAAACGGAAGTCACGGTTCAGTTGATTAACTATACGAAGAGCGGAAAAAAATTCTGGAATTTATTTCACTTGCAGCCTATGCGTGACCAGAAGGGCGATGTCCAGTATTTCATTGGCGTTCAGCTTGATGGTACCGAGCATGTTCGCGATGCTGCGGAGCGTGAAGGTGTAATGTTAATTAAAAAGACTGCTGAAAACATTGATGAAGCGGCCAAAGGGAGCCTGCATCCGCCGATGGATAACCGCGTG -n RL1_crispresso_result --guide_seq GAAGCGATTAACGAAAGCAGCGGT,TGAAAACATTGATGAAGCGGCCAAA --cleavage_offset 0

In [7]:
### define the necessary variables

base_dir = os.getcwd() 
amplicon = "RRMEAINESSGLATTLERIEKNFVITDPRLPDNPIIFASDSFLQLTEYSREEILGRNCRFLQGPETDRATVRKIRDAIDNQTEVTVQLINYTKSGKKFWNLFHLQPMRDQKGDVQYFIGVQLDGTEHVRDAAEREGVMLIKKTAENIDEAAKGSLHPPMDNRV"

amplicon_DNA = 'CGCCGCATGGAAGCGATTAACGAAAGCAGCGGTTTAGCCACAACGCTGGAACGCATTGAAAAGAATTTCGTAATCACAGACCCGCGCCTTCCCGACAATCCAATTATTTTTGCGTCCGATAGCTTCCTGCAATTAACCGAATACAGCCGCGAAGAAATTCTGGGTCGTAATTGTCGCTTCCTTCAGGGGCCAGAGACTGACCGTGCTACGGTACGCAAAATCCGCGACGCAATCGACAATCAAACGGAAGTCACGGTTCAGTTGATTAACTATACGAAGAGCGGAAAAAAATTCTGGAATTTATTTCACTTGCAGCCTATGCGTGACCAGAAGGGCGATGTCCAGTATTTCATTGGCGTTCAGCTTGATGGTACCGAGCATGTTCGCGATGCTGCGGAGCGTGAAGGTGTAATGTTAATTAAAAAGACTGCTGAAAACATTGATGAGGCGGCCAAAGGGAGCCTGCATCCGCCGATGGATAACCGCGTG'

amplicon_DNA_corrected = 'CGCCGCATGGAAGCGATTAACGAAAGCAGCGGTTTAGCCACAACGCTGGAACGCATTGAAAAGAATTTCGTAATCACAGACCCGCGCCTTCCCGACAATCCAATTATTTTTGCGTCCGATAGCTTCCTGCAATTAACCGAATACAGCCGCGAAGAAATTCTGGGTCGTAATTGTCGCTTCCTTCAGGGGCCAGAGACTGACCGTGCTACGGTACGCAAAATCCGCGACGCAATCGACAATCAAACGGAAGTCACGGTTCAGTTGATTAACTATACGAAGAGCGGAAAAAAATTCTGGAATTTATTTCACTTGCAGCCTATGCGTGACCAGAAGGGCGATGTCCAGTATTTCATTGGCGTTCAGCTTGATGGTACCGAGCATGTTCGCGATGCTGCGGAGCGTGAAGGTGTAATGTTAATTAAAAAGACTGCTGAAAACATTGATGAAGCGGCCAAAGGGAGCCTGCATCCGCCGATGGATAACCGCGTG'## since all the sequenced reads have an "A" instead of "G" at the 413th position of the LOV2 gene, we correct the sequence here

amplicon_start = "CGCCGCATGGAAGCGATTAACGAAAGCAGCGGT"
amplicon_end = "GGGAGCCTGCATCCGCCGATGGATAACCGCGTG"
LOV_gene_end = "ACATTGATGAAGCGGCCAAA"

LOV_gene = "TTAGCCACAACGCTGGAACGCATTGAAAAGAATTTCGTAATCACAGACCCGCGCCTTCCCGACAATCCAATTATTTTTGCGTCCGATAGCTTCCTGCAATTAACCGAATACAGCCGCGAAGAAATTCTGGGTCGTAATTGTCGCTTCCTTCAGGGGCCAGAGACTGACCGTGCTACGGTACGCAAAATCCGCGACGCAATCGACAATCAAACGGAAGTCACGGTTCAGTTGATTAACTATACGAAGAGCGGAAAAAAATTCTGGAATTTATTTCACTTGCAGCCTATGCGTGACCAGAAGGGCGATGTCCAGTATTTCATTGGCGTTCAGCTTGATGGTACCGAGCATGTTCGCGATGCTGCGGAGCGTGAAGGTGTAATGTTAATTAAAAAGACTGCTGAAAACATTGATGAGGCGGCCAAA"

LOV_gene_corrected = 'TTAGCCACAACGCTGGAACGCATTGAAAAGAATTTCGTAATCACAGACCCGCGCCTTCCCGACAATCCAATTATTTTTGCGTCCGATAGCTTCCTGCAATTAACCGAATACAGCCGCGAAGAAATTCTGGGTCGTAATTGTCGCTTCCTTCAGGGGCCAGAGACTGACCGTGCTACGGTACGCAAAATCCGCGACGCAATCGACAATCAAACGGAAGTCACGGTTCAGTTGATTAACTATACGAAGAGCGGAAAAAAATTCTGGAATTTATTTCACTTGCAGCCTATGCGTGACCAGAAGGGCGATGTCCAGTATTTCATTGGCGTTCAGCTTGATGGTACCGAGCATGTTCGCGATGCTGCGGAGCGTGAAGGTGTAATGTTAATTAAAAAGACTGCTGAAAACATTGATGAAGCGGCCAAA' ## since all the sequenced reads have an "A" instead of "G" at the 413th position, we correct the sequence here

LOV = "LATTLERIEKNFVITDPRLPDNPIIFASDSFLQLTEYSREEILGRNCRFLQGPETDRATVRKIRDAIDNQTEVTVQLINYTKSGKKFWNLFHLQPMRDQKGDVQYFIGVQLDGTEHVRDAAEREGVMLIKKTAENIDEAAK"

gene_len = len(LOV)*3
catch_left = 'AAGCAGCGGT'
catch_right = 'GGGAGCCTGC'
offset_left = 3
offset_right= 5

genetic_code = {
  'ATA': 'I', 'ATC': 'I', 'ATT': 'I', 'ATG': 'M',
  'ACA': 'T', 'ACC': 'T', 'ACG': 'T', 'ACT': 'T',
  'AAC': 'N', 'AAT': 'N', 'AAA': 'K', 'AAG': 'K',
  'AGC': 'S', 'AGT': 'S', 'AGA': 'R', 'AGG': 'R',
  'CTA': 'L', 'CTC': 'L', 'CTG': 'L', 'CTT': 'L',
  'CCA': 'P', 'CCC': 'P', 'CCG': 'P', 'CCT': 'P',
  'CAC': 'H', 'CAT': 'H', 'CAA': 'Q', 'CAG': 'Q',
  'CGA': 'R', 'CGC': 'R', 'CGG': 'R', 'CGT': 'R',
  'GTA': 'V', 'GTC': 'V', 'GTG': 'V', 'GTT': 'V',
  'GCA': 'A', 'GCC': 'A', 'GCG': 'A', 'GCT': 'A',
  'GAC': 'D', 'GAT': 'D', 'GAA': 'E', 'GAG': 'E',
  'GGA': 'G', 'GGC': 'G', 'GGG': 'G', 'GGT': 'G',
  'TCA': 'S', 'TCC': 'S', 'TCG': 'S', 'TCT': 'S',
  'TTC': 'F', 'TTT': 'F', 'TTA': 'L', 'TTG': 'L',
  'TAC': 'Y', 'TAT': 'Y', 'TAA': '*', 'TAG': '*',
  'TGC': 'C', 'TGT': 'C', 'TGA': '*', 'TGG': 'W',
}

codons = list(genetic_code.keys())


quality_score = {
  '!':0, '"':1, '#':2, '$':3, '%':4, '&':5, "'":6, '(':7, ')':8, '*':9,
  '+':10, ',':11, '-':12, '.':13, '/':14, '0':15, '1':16, '2':17, '3':18, '4':19,
  '5':20, '6':21, '7':22, '8':23, '9':24, ':':25, ';':26, '<':27, '=':28, '>':29,
  '?':30, '@':31, 'A':32, 'B':33, 'C':34, 'D':35, 'E':36, 'F':37, 'G':38, 'H':39, 'I':40
}


ecoli_pref = { ### codons used for retron library (RL8) construction
            "A": 'GCG',
            "R": 'CGT',
            "N": 'AAC',
            "D": 'GAT',
            "C": 'TGC',
            "Q": 'CAG',
            "E": 'GAA',
            "G": 'GGC',
            "H": 'CAT',
            "I": 'ATT',
            "L": "CTG",
            "K": 'AAA',
            "M": 'ATG',
            "F": "TTT",
            "P": 'CCG',
            "S": 'AGC',
            "T": 'ACC',
            "W": 'TGG',
            "Y": "TAT",
            "V": 'GTG',
}

In [227]:
insertions = [#"X",
   #"GG",
   # "PP",
    #"GXG"
    "GGSG",
    "GSGG",
    "GPPG",
    "GSGSG",
    "GPPPG",
    "GSGGSG",
    "GPPPPG"] +["G"+x+"G" for x in ecoli_pref.keys()] # + "GXG" AAs ## !!! "X" insertions are not included here



In [228]:
insertion_codons = {insertion : "".join([ecoli_pref[Aa] for Aa in insertion]) for insertion in insertions }
subst = {'GG': 'GGCGGC',
 'PP': 'CCGCCG'}
deletions = {"del1": "-"*9, # deletion of SG + deletion of the AA in front = total of 9 nucleotides
             "del2": "-"*12,
             "del3": "-"*15,}

In [196]:
## import data set from CRISPResso2 analysis

Allele_frequency = pd.read_csv("data/fastq/CRISPResso_on_RL1-R1_crispresso_result/Alleles_frequency_table_around_sgRNA_GAAGCGATTAACGAAAGCAGCGGT.txt", sep = "\t")
# filter for alleles with a frequency > 0.01
Allele_frequency = Allele_frequency[Allele_frequency["%Reads"] > 0.1]

In [261]:
indel_codons_count = {}

## in a few cases, four deletions but one insertion is defined, for which we think the read is wrongly aligned and which is only a deletion of 3 nucleotides with a mutation
## thus we combine # insertion - # deletion to define the size difference to the reference seq
Allele_frequency["size_diff"] =  Allele_frequency["n_inserted"] - Allele_frequency["n_deleted"]

# we now move all deletions to the end of the read 
## first we cut all the reads at the beginning of the LOV gene, thereby only keeping the linker sequence
LOV2_start = Allele_frequency.iloc[0,0].find("TTAGCCAC")
Allele_frequency["linker_seq"] = Allele_frequency.iloc[:,0].apply(lambda x: x[:LOV2_start])
## now we move all deletions to the end of the read
Allele_frequency["linker_sequence"] =  Allele_frequency.loc[:,"linker_seq"].apply(lambda x: x.replace("-","")) + ["-"*-x for x in Allele_frequency["size_diff"]]

linker_sequences_freq = Allele_frequency[["linker_sequence", "%Reads", "size_diff"]] 

### indel counts: 
## we can not count "X" indels directly, since these codons are present anyways in the reference sequence, thus we first need to select for deletion size = 3 
linker_reads_indelX = linker_sequences_freq[[seq.count("-") == 3 for seq in linker_sequences_freq["linker_sequence"]]].copy()
linker_reads_indelX["indelX"] = [seq[-6:-3] for seq in linker_reads_indelX["linker_sequence"]]
for idx, indel_codon in enumerate(linker_reads_indelX["indelX"]):
    indel_codons_count[indel_codon] = linker_reads_indelX.iloc[idx,1]

print("intended X substitutions", sum([x in ecoli_pref.values() for x in linker_reads_indelX["indelX"]])) 
print("unintended X substitutions", sum([x not in ecoli_pref.values() for x in linker_reads_indelX["indelX"]])) ## test whether these X subst were intended (i.e. they are part of our e coli references) --> = 0 so we can count them as intended X insertions


##search for substitutions only (GG or PP instead of SG)
linker_reads_subst = linker_sequences_freq[linker_sequences_freq["size_diff"] == 0].copy()
for codon in subst.values():
    for idx,seq in enumerate(linker_reads_subst["linker_sequence"]):
        if codon in seq:
            indel_codons_count[codon] = linker_reads_subst.iloc[idx,1]

## search for indels
linker_reads_indels = linker_sequences_freq[linker_sequences_freq["size_diff"] >1].copy()

for insertion_codon in insertion_codons.values():
    for idx,seq in enumerate(linker_reads_indels["linker_sequence"]):
        if insertion_codon in seq:
            indel_codons_count[insertion_codon] = linker_reads_indels.iloc[idx,1] 


# search for deletions
linker_reads_deletions = linker_sequences_freq[linker_sequences_freq["size_diff"] <-6].copy()

for deletion in deletions.values():
    for idx,seq in enumerate(linker_reads_deletions["linker_sequence"]):
        if deletion.count("-") == seq.count("-"):
            indel_codons_count[deletion] = linker_reads_deletions.iloc[idx,1]

intended X substitutions 20
unintended X substitutions 0


In [262]:
linker_reads_deletions

,linker_sequence,%Reads,size_diff
41,GGAAGCGATTAAC------------,0.149744,-12
46,GGAAGCGATT---------------,0.123702,-15


In [251]:
all_indels = {**insertion_codons, ** subst, ** deletions, ** ecoli_pref}

In [265]:
indel_linker_count = {translate_dna2aa(indel) if "-" not in indel else indel: value for indel, value in indel_codons_count.items()}

In [266]:
indel_linker_count

{'Q': np.float64(0.3841270874702953),
 'S': np.float64(0.3841270874702953),
 'V': np.float64(0.3548292587649337),
 'W': np.float64(0.3548292587649337),
 'C': np.float64(0.3450633158631466),
 'L': np.float64(0.3222761157589765),
 'P': np.float64(0.3190208014583808),
 'R': np.float64(0.3125101728571893),
 'A': np.float64(0.3092548585565936),
 'T': np.float64(0.3027442299554022),
 'G': np.float64(0.2897229727530193),
 'F': np.float64(0.2636804583482535),
 'M': np.float64(0.2604251440476578),
 'I': np.float64(0.2441485725446791),
 'E': np.float64(0.2408932582440834),
 'H': np.float64(0.2246166867411048),
 'N': np.float64(0.2213613724405091),
 'Y': np.float64(0.2083401152381262),
 'D': np.float64(0.2050848009375305),
 'K': np.float64(0.1725316579315733),
 'GG': np.float64(0.2115954295387219),
 'PP': np.float64(0.117191314821446),
 'GPPPG': np.float64(0.117191314821446),
 'GAG': np.float64(0.1562550864285946),
 'GRG': np.float64(0.1334678863244246),
 'GNG': np.float64(0.1464891435268075),
 '